<a href="https://colab.research.google.com/github/Mike030668/MIPT_magistratura/blob/main/NLP_generation/Generation/notebooks/Telegram_tyni_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Устанавливаем необходимый пакет*

Для взаимодействия с Telegram API средствами языка Python разработаны несколько модулей. Самый простой из них — Telebot.


In [1]:
!git clone https://github.com/Mike030668/MIPT_magistratura.git -q
!pip3 install pytelegrambotapi -q
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install bitsandbytes einops wandb -Uqqq

import gdown  # библиотека по работе с файлами в том числе и с гугл_диска
import pandas as pd
import numpy as np
import random
import torch
import gc
from transformers import AutoTokenizer, AutoModel
import sys

from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig

import warnings
warnings.filterwarnings('ignore')


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAIN_DIR =  "/content/MIPT_magistratura/NLP_generation/Generation"

sys.path.append(MAIN_DIR)
from utils.download import gd_folder_download
from utils.constant import SHARE_DIR, TOKEN
from utils.generate import generate_answer, get_prompt, ROLES


# Подгружаем с публичной папки по https
dir_to = MAIN_DIR + "/TinyLamma/"

# код потребует гугл авторизацию любую из имеющихся
model_dir = gd_folder_download(SHARE_DIR, dir_to)

config = PeftConfig.from_pretrained(model_dir)
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	quantization_config=bnb_config,
	trust_remote_code=True,
	device_map='auto'

	)

# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

1IIXHWzZbftyD1NOOGhb0e2tvE9lER7A- README.md text/markdown (1/12)
Download 100%.


1IdiVQuJ8h5IvJqQM887p92JtIVnBvuqi adapter_config.json application/json (2/12)
1IezAG4VYQGH_U3jOj9i9fWTI84IgpJcE adapter_model.safetensors application/octet-stream (3/12)
1IJgvo7LR3w8lyn-hRVPyBFgkv1gqHO6G optimizer.pt application/x-zip (4/12)
Download 100%.


1IkYI5lnMPCdRTJls8mq_uAFHRA86CN32 rng_state.pth application/x-zip (5/12)
Download 100%.


1InSO93MkaLOQ11CZVVOHD_Df-TRrjVMa scheduler.pt application/x-zip (6/12)
Download 100%.

...

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [2]:
import telebot
from telebot import types

# Создаем экземпляр бота
bot = telebot.TeleBot(TOKEN)

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    # Добавляем две кнопки
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    for role in ROLES:
      item=types.KeyboardButton(role)
      markup.add(item)

    bot.send_message(m.chat.id, 'Бот запущен. Начните общение')

conext_memory = ''
get_role = False
# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    global conext_memory, get_role

    print("in_conext_memory", conext_memory)
    print()

    if not get_role:
        for role in ROLES:
          print("message.text", message.text)
          print(role)
          if role in message.text:
            print("get_role ", role)
            get_role = role
            break

        if not get_role: get_role = "Kyle"

    if get_role:
       prompt = get_prompt(query = message.text, context = conext_memory, role = get_role)
       print("have role" , get_role)


    gen_answer = generate_answer(prompt, model, tokenizer, max_new_tokens = 250, device = DEVICE)

    conext_memory = '.'.join([conext_memory, message.text, gen_answer])

    print("gen_answer", gen_answer)
    print()
    print("out_conext_memory", conext_memory)
    # Отсылаем сообщение в чат пользователя
    try:
      bot.send_message(message.chat.id, gen_answer)
    except: pass
# Запускаем бота

bot.polling(none_stop=True, interval=0)

in_conext_memory 

message.text Chao Cartman
Stan
message.text Chao Cartman
Kyle
message.text Chao Cartman
Cartman
get_role  Cartman
have role Cartman


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


gen_answer  a question.

out_conext_memory .Chao Cartman. a question.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory .Chao Cartman. a question.

have role Cartman
gen_answer  the answer of the user, and then use it as a parameter for the next step.

out_conext_memory .Chao Cartman. a question..How are you Cartman?. the answer of the user, and then use it as a parameter for the next step.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory .Chao Cartman. a question..How are you Cartman?. the answer of the user, and then use it as a parameter for the next step.

have role Cartman
gen_answer  the parameters that we have defined so far.

out_conext_memory .Chao Cartman. a question..How are you Cartman?. the answer of the user, and then use it as a parameter for the next step..let's talk about live. the parameters that we have defined so far.


In [ ]:
"Cartman" in "you are Cartman"

True